In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

# cleaning

In [ ]:
protein_one = pd.read_csv('/Users/jonathanfarro/Desktop/animal-protein-consumption.csv')

life_ex = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/life-expectancy-hmd-unwpp.csv')

usa_pricing = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/average_prices_for_food.csv')

daily_cals = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-caloric-supply-derived-from-carbohydrates-protein-and-fat.csv')

fruit_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/fruit-consumption-per-capita.csv')

veg_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/vegetable-consumption-per-capita/vegetable-consumption-per-capita.csv' )

nutrition =  pd.read_excel('/Users/jonathanfarro/Desktop/ Capstone Files/food_comp.xlsx')

protein_per_GDP = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-per-capita-protein-supply-vs-gdp-per-capita.csv')

death = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/death-rate-by-cause-who-mdb.csv')



In [ ]:
# rename columns to something easier to work with 

protein = protein_one.rename(columns={
                        protein_one.columns[3]:'fish',
                        protein_one.columns[4]:'poultry',
                        protein_one.columns[5]:'pork',
                        protein_one.columns[6]:'beef',
                        protein_one.columns[7]:'sheep/goat',
                        protein_one.columns[8]:'other',
                        protein_one.columns[9]:'eggs',
                        protein_one.columns[10]:'milk'
                       })

In [ ]:
# bulk clean up 
protein = protein.rename(columns=lambda x: x.lower())
protein = protein.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)

life_ex = life_ex.rename(columns = lambda x: x.lower())
life_ex = life_ex.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
life_ex = life_ex.rename(columns = {life_ex.columns[3] : 'life_expectancy(0)'})

protein_per_GDP = protein_per_GDP.rename(columns = lambda x: x.lower())
protein_per_GDP = protein_per_GDP.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
gdp_population = protein_per_GDP.drop(columns= {protein_per_GDP.columns[3],protein_per_GDP.columns[6]})
gdp_population = gdp_population.rename(columns = { gdp_population.columns[3]: 'gdp_per_capita'}) 


daily_cals = daily_cals.rename(columns = lambda x: x.lower())
daily_cals = daily_cals.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

death = death.rename(columns = lambda x: x.lower())
death = death.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

nutrition = nutrition.rename(columns = lambda x: x.lower())
nutrition = nutrition.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

fruit_per_cap = fruit_per_cap.rename(columns=lambda x: x.lower())
fruit_per_cap = fruit_per_cap.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)
fruit = fruit_per_cap.rename(columns = {fruit_per_cap.columns[3]:'fruit_kilo_yearly'}).reset_index(drop = True)


In [ ]:
# creating a new column to show total_protein intake for a day.
protein['total_protein'] = protein.iloc[:, 3:11].sum(axis = 1).round(2)

In [ ]:
fruit_per_cap = fruit_per_cap.rename(columns=lambda x: x.lower())
fruit_per_cap = fruit_per_cap.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)
fruit = fruit_per_cap.rename(columns = {fruit_per_cap.columns[3]:'fruit_kilo_yearly'}).reset_index(drop = True)

In [ ]:
capstone = pd.merge(protein,
         life_ex,
         on= ['entity','code','year'],
         how= 'inner')

capstone.head()

In [ ]:
capstone_2 = pd.merge( capstone, gdp_population, on = ['entity','code','year'],how= 'inner')

main_frame = pd.merge(capstone_2,death,  on = ['entity','code','year'],how= 'outer')

final_frame = pd.merge(main_frame, daily_cals,  on = ['entity','code','year'],how= 'outer')

final_frame['total_daily_protein']=final_frame.iloc[:, 3:10].sum(axis=1)

final_frame = final_frame.rename(columns= { 'daily calorie supply per person that comes from animal protein':'daily_cal_animal_protein',
                              'daily calorie supply per person that comes from vegetal protein':'daily_cal_veg_protein',
                              'daily calorie supply per person from fat':'daily_cal_fat',
                              'daily calorie supply per person from carbohydrates':'daily_cal_carbs'})
final_frame.entity.unique()

In [ ]:
#creating an all_cause_mortality column, and more cleaning

In [ ]:
final_frame['all_cause_per_100k'] = final_frame.iloc[:,15:24].sum(axis=1)

final_frame_two = final_frame.drop(columns=final_frame.columns[15:24])

final_frame_three = final_frame_two[~final_frame_two['entity'].str.contains(r'\(.*\)')]

final = final_frame_three
final = final.rename(columns={'life_expectancy(0)':'life_exp','population (historical)':'population'})

In [ ]:
#further cleaning.

non_countries_list = ['asia','europe','high-income countries','low-income countries','lower-middle-income countries','oceania','upper-middle-income countries','world']
income_classes= ['low-income countries','high-income countries','lower-middle-income countries','upper-middle-income countries','world']

final_countries = final.loc[~final['entity'].isin(non_countries_list)]
income_class = final.loc[final['entity'].isin(income_classes)]

testdrop= final_countries

df = testdrop.dropna()

df['cals_from_protein'] = df['daily_cal_animal_protein'] + df['daily_cal_veg_protein']



# creating regions 

In [ ]:
country_to_region = {
    'algeria': 'Africa',
    'angola': 'Africa',
    'benin': 'Africa',
    'botswana': 'Africa',
    'burkina faso': 'Africa',
    'burundi': 'Africa',
    'cabo verde': 'Africa',
    'cameroon': 'Africa',
    'central african republic': 'Africa',
    'chad': 'Africa',
    'comoros': 'Africa',
    'congo, democratic republic of the': 'Africa',
    'congo, republic of the': 'Africa',
    "côte d'ivoire": 'Africa',
    'djibouti': 'Africa',
    'egypt': 'Africa',
    'equatorial guinea': 'Africa',
    'eritrea': 'Africa',
    'eswatini': 'Africa',
    'ethiopia': 'Africa',
    'gabon': 'Africa',
    'gambia': 'Africa',
    'ghana': 'Africa',
    'guinea': 'Africa',
    'guinea-bissau': 'Africa',
    'kenya': 'Africa',
    'lesotho': 'Africa',
    'liberia': 'Africa',
    'libya': 'Africa',
    'madagascar': 'Africa',
    'malawi': 'Africa',
    'mali': 'Africa',
    'mauritania': 'Africa',
    'mauritius': 'Africa',
    'morocco': 'Africa',
    'mozambique': 'Africa',
    'namibia': 'Africa',
    'niger': 'Africa',
    'nigeria': 'Africa',
    'rwanda': 'Africa',
    'sao tome and principe': 'Africa',
    'senegal': 'Africa',
    'seychelles': 'Africa',
    'sierra leone': 'Africa',
    'somalia': 'Africa',
    'south africa': 'Africa',
    'south sudan': 'Africa',
    'sudan': 'Africa',
    'tanzania': 'Africa',
    'togo': 'Africa',
    'tunisia': 'Africa',
    'uganda': 'Africa',
    'zambia': 'Africa',
    'zimbabwe': 'Africa',
    'afghanistan': 'Asia',
    'armenia': 'Asia',
    'azerbaijan': 'Asia',
    'bahrain': 'Asia',
    'bangladesh': 'Asia',
    'bhutan': 'Asia',
    'brunei': 'Asia',
    'cambodia': 'Asia',
    'china': 'Asia',
    'cyprus': 'Asia', 
    'georgia': 'Asia',
    'india': 'Asia',
    'indonesia': 'Asia',
    'iran': 'Asia',
    'iraq': 'Asia',
    'israel': 'Asia',
    'japan': 'Asia',
    'jordan': 'Asia',
    'kazakhstan': 'Asia',
    'kuwait': 'Asia',
    'kyrgyzstan': 'Asia',
    'laos': 'Asia',
    'lebanon': 'Asia',
    'malaysia': 'Asia',
    'maldives': 'Asia',
    'mongolia': 'Asia',
    'myanmar': 'Asia',
    'nepal': 'Asia',
    'north korea': 'Asia',
    'oman': 'Asia',
    'pakistan': 'Asia',
    'palestine': 'Asia',
    'philippines': 'Asia',
    'qatar': 'Asia',
    'russia': 'Asia', # Mostly in Asia
    'saudi arabia': 'Asia',
    'singapore': 'Asia',
    'south korea': 'Asia',
    'sri lanka': 'Asia',
    'syria': 'Asia',
    'tajikistan': 'Asia',
    'thailand': 'Asia',
    'timor-leste': 'Asia',
    'turkey': 'Asia', # Mostly in Asia
    'turkmenistan': 'Asia',
    'united arab emirates': 'Asia',
    'uzbekistan': 'Asia',
    'vietnam': 'Asia',
    'yemen': 'Asia',
    'albania': 'Europe',
    'andorra': 'Europe',
    'austria': 'Europe',
    'belarus': 'Europe',
    'belgium': 'Europe',
    'bosnia and herzegovina': 'Europe',
    'bulgaria': 'Europe',
    'croatia': 'Europe',
    'czech republic': 'Europe',
    'denmark': 'Europe',
    'estonia': 'Europe',
    'finland': 'Europe',
    'france': 'Europe',
    'germany': 'Europe',
    'greece': 'Europe',
    'hungary': 'Europe',
    'iceland': 'Europe',
    'ireland': 'Europe',
    'italy': 'Europe',
    'kosovo': 'Europe',
    'latvia': 'Europe',
    'liechtenstein': 'Europe',
    'lithuania': 'Europe',
    'luxembourg': 'Europe',
    'malta': 'Europe',
    'moldova': 'Europe',
    'monaco': 'Europe',
    'montenegro': 'Europe',
    'netherlands': 'Europe',
    'north macedonia': 'Europe',
    'norway': 'Europe',
    'poland': 'Europe',
    'portugal': 'Europe',
    'romania': 'Europe',
    'russia': 'Europe', 
    'san marino': 'Europe',
    'serbia': 'Europe',
    'slovakia': 'Europe',
    'slovenia': 'Europe',
    'spain': 'Europe',
    'sweden': 'Europe',
    'switzerland': 'Europe',
    'ukraine': 'Europe',
    'united kingdom': 'Europe',
    'vatican city': 'Europe', 
    'antigua and barbuda': 'North America',
    'bahamas': 'North America',
    'barbados': 'North America',
    'belize': 'North America',
    'canada': 'North America',
    'costa rica': 'North America',
    'cuba': 'North America',
    'dominica': 'North America',
    'dominican republic': 'North America',
    'el salvador': 'North America',
    'grenada': 'North America',
    'guatemala': 'North America',
    'haiti': 'North America',
    'honduras': 'North America',
    'jamaica': 'North America',
    'mexico': 'North America',
    'nicaragua': 'North America',
    'panama': 'North America',
    'saint kitts and nevis': 'North America',
    'saint lucia': 'North America',
    'saint vincent and the grenadines': 'North America',
    'trinidad and tobago': 'North America',
    'united states': 'North America',
    'australia': 'Oceania',
    'fiji': 'Oceania',
    'kiribati': 'Oceania',
    'marshall islands': 'Oceania',
    'micronesia': 'Oceania',
    'nauru': 'Oceania',
    'new zealand': 'Oceania',
    'palau': 'Oceania',
    'papua new guinea': 'Oceania',
    'samoa': 'Oceania',
    'solomon islands': 'Oceania',
    'tonga': 'Oceania',
    'tuvalu': 'Oceania',
    'vanuatu': 'Oceania',
    'argentina': 'South America',
    'bolivia': 'South America',
    'brazil': 'South America',
    'chile': 'South America',
    'colombia': 'South America',
    'ecuador': 'South America',
    'guyana': 'South America',
    'paraguay': 'South America',
    'peru': 'South America',
    'suriname': 'South America',
    'uruguay': 'South America',
    'venezuela': 'South America',
}

df['region'] = df['entity'].map(country_to_region)

df[['entity','region']]

# correlation

In [ ]:
correlations = df.iloc[:,2:21].corr()
correlations

In [ ]:
df.year.sort_values().unique()

# how has protein consumption changed over the years? 

In [ ]:
# you can see a conintual increase in animal protein, and protein in general going up as the life expectancy increases.

In [ ]:
correlating_food_type_and_life = df.groupby('year').agg({'life_exp':'mean','daily_cal_veg_protein':'mean','daily_cal_animal_protein':'mean','population':'sum'})

In [ ]:
plt.figure(figsize=(10, 6.5)) 
sns.barplot(data= correlating_food_type_and_life,x='year',y='daily_cal_veg_protein',color='g',alpha=0.5,label="Vegetable Protein")
sns.barplot(data= correlating_food_type_and_life,x='year',y='daily_cal_animal_protein',color='orange',alpha=0.5,label="Animal Protein")
sns.barplot(data= correlating_food_type_and_life,x='year',y='life_exp',color='brown',label="Life Expectancy",alpha=0.8)

plt.xticks(rotation=35)
plt.xlabel('')
plt.ylabel('');

In [ ]:
sns.set_color_codes("muted")
plt.figure(figsize=(10, 5)) 
sns.lineplot(x=df.year ,y=df['daily_cal_veg_protein'],color='g',label="Vegetable Protein")
sns.lineplot(x=df.year, y=df['daily_cal_animal_protein'],color='r',label="Animal Protein")
plt.xlabel('')
plt.ylabel('');

In [ ]:
sns.set_color_codes("muted")
plt.figure(figsize=(8, 6))
sns.color_palette("Spectral", as_cmap=True)
sns.barplot(df,x='daily_cal_veg_protein',y='year',orient='h',alpha=0.7,color='g',label="Vegetable Protein")
sns.barplot(df,x='daily_cal_animal_protein',y='year',orient='h',alpha=0.7,color='r',label="Animal Protein")
plt.legend(bbox_to_anchor=(0, 1.02, .6, 0.102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
plt.xlabel('Protein Grams Per Day')
plt.ylabel('');

#  Daily protein per capita in grams consumption over the years and life expectancy.

In [ ]:
life_time = df.groupby('year').agg({'life_exp':'mean'}).sort_values(by='life_exp') 
life_time = life_time.sort_values(by='year')

In [ ]:
sns.set_color_codes("muted")
plt.figure(figsize=(15, 6))
sns.barplot(df,x='year',y='life_exp',color='g',alpha=0.5,label= "Life Expectancy From Birth")
sns.barplot(df,x='year',y='total_daily_protein',color='r',alpha=0.5,label="Total Daily Protein")


plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation = 35);

  # total calories derived from veg vs animal protein 

# #4.)  countries who derive protein mainly from animal sources.

In [ ]:
animal_protein = df.groupby(['region','year']).agg({
    'cals_from_protein':'sum'}).sort_values(by='cals_from_protein',ascending = False).reset_index()
plt.figure(figsize=(15, 5)) 
sns.scatterplot(df,x = 'year',y='cals_from_protein',legend=False,hue='region')

plt.xlabel('')

plt.ylabel(' Protein Consumption');


In [ ]:
animal_heavy_list = animal_protein.region.to_list()
animal_protein

###  countries who derive protein from mainly veg.

In [ ]:
veg_pro = df.groupby(['entity','year']).agg({
    'daily_cal_veg_protein':'sum'}).sort_values(by='daily_cal_veg_protein',ascending = False).reset_index()


veg_pro

In [ ]:
veg_heavy_list = veg_pro.entity.to_list()

top_veg_protein = df.loc[df['entity'].isin(veg_heavy_list)].reset_index(drop=True)


In [ ]:
sns.scatterplot(veg_pro,x = 'year',y='daily_cal_veg_protein',hue='entity',legend=False)
plt.xlabel('')
plt.ylabel('Veg Protein Consumption');

# life expectancy charts

 ##### There is a strong positive correlation of 0.78 between calories derived from animal protein and a weak positive
 ##### correlation of 0.022 between calories derived from veg protein.

In [ ]:

plt.figure(figsize=(12, 6))  
sns.scatterplot(data= df , x='year', y='life_exp',legend=False)  
plt.xlabel('')
plt.ylabel('Life Expectancy')
plt.title('Life Expectancy by Year')
plt.tight_layout()
plt.show()

# #7.)how does life expectancy compare beteween countries that derive calories from protein vs carbs vs fat?  

In [ ]:
 # sorted carbohydrate group 

carb_heavy = df.groupby(['region']).agg({
                                            'daily_cal_carbs':'max',
                                            'life_exp':'max',
                                            'gdp_per_capita':'mean'  
                                            }).sort_values(by='daily_cal_carbs',ascending = False).reset_index()

carb_heavy['most_calorie_source'] = "High Carb Diet"
carb_heavy

In [ ]:
#fat group

fat_cals = df.groupby(['region']).agg({
    'daily_cal_fat':'max',
    'life_exp':'max',
    'gdp_per_capita':'mean'
    }).sort_values(by='daily_cal_fat',ascending = False).reset_index()

fat_cals['most_calorie_source']= "High Fat Diets"
fat_cals

In [ ]:
#protein group

top_three_protein_cals = df.groupby('region').agg({'cals_from_protein':'max','life_exp':'max','gdp_per_capita':'mean'}).sort_values(by='cals_from_protein',ascending = False)

top_three_protein_cals = top_three_protein_cals.reset_index()
top_three_protein_cals['most_calorie_source']= "High Protein Diets"
high_protein_diets = top_three_protein_cals

In [ ]:
#concat 

seven1 = pd.concat([carb_heavy,fat_cals])

seven_concat = pd.concat([seven1,high_protein_diets])

seven_life = seven_concat.sort_values(by='life_exp',ascending = False)
seven_gdp  = seven_concat.sort_values(by='gdp_per_capita',ascending = False)

sns.barplot(seven_concat.sort_values(by='life_exp',ascending=False) ,x='life_exp',y='gdp_per_capita',hue='region')

In [ ]:
plt.figure(figsize=(12, 8))

# Assuming 'seven_life' and 'seven_gdp' are your DataFrames

#  Create a color mapping dictionary
unique_categories = seven_concat['most_calorie_source'].unique() 
color_map = {category: plt.cm.Set3(i) for i, category in enumerate(unique_categories)}

#  Plot life expectancy with color mapping
for i, row in seven_life.iterrows():
    plt.bar(row['region'], row['life_exp'], color=color_map[row['most_calorie_source']], alpha=1)


# Plot GDP per capita with color mapping (assuming 'most_calorie_source' exists in 'seven_gdp' too)
for i, row in seven_gdp.iterrows():
    plt.bar(row['region'], row['gdp_per_capita'], color=color_map[row['most_calorie_source']], alpha=0.4)
plt.scatter(seven_gdp['region'],seven_gdp['gdp_per_capita'], color='gold', marker=(8, 2, 0), s=150, label='GDP per Capita')
plt.scatter(seven_life['region'], seven_life['life_exp'], color='moccasin', marker=(8, 2, 0), s=150, label='Life Expectancy')
legend1 = plt.legend(bbox_to_anchor=(0.187, 0.85), loc='upper right')  # Legend for scatter plots

plt.yscale('log')
plt.ylabel('')
plt.xlabel('')

# Add the legend
plt.gca().add_artist(legend1) 
plt.legend()

# (Optional) Add a color legend
handles = [plt.Rectangle((0, 0), 1, 1, color=color_map[category]) for category in unique_categories]
plt.legend(handles, unique_categories, title='Most Calorie Source', loc='upper left') 
plt.yscale('log')
plt.gca().get_yaxis().set_visible(False) 
plt.show()

In [ ]:
# while there is a correlation betweeen life expectancy and the type of foods you eat, this graph clearly shows the stronger correlation to gdp and what you eat.

# who in our world, on average, lives the longest/shortest and what are they eating? 

In [ ]:
grouped = df.groupby('entity')

# Define conditions
condition1 = df['life_exp'] > df['life_exp'].mean()


# Apply .loc with conditions to each group
life_exp_above_average = grouped.apply(lambda x: x.loc[condition1 ])
sns.scatterplot(life_exp_above_average,x='year',y='life_exp')

In [ ]:
avg_life_exp = df.groupby(['entity']) \
                                   .agg({'life_exp': 'mean'}) \
                                   .reset_index() 

avg_life_exp = avg_life_exp.sort_values(by='life_exp',ascending = False).reset_index(drop=True)

top_3_life_exp = avg_life_exp
bottom_life_exp = avg_life_exp

In [ ]:
long_life_list = top_3_life_exp .entity.to_list()


In [ ]:
long_life = final.loc[final['entity'].isin(long_life_list)]


In [ ]:
long_life = long_life.groupby('entity').agg({'life_exp':'mean','total_daily_protein':'mean','daily_cal_animal_protein':'mean','daily_cal_veg_protein':'mean','gdp_per_capita':'mean','year':'count'}).reset_index().round(2)

In [ ]:
short_list = df.groupby(['entity']) \
                                   .agg({'life_exp': 'max'}) \
                                   .sort_values(by='life_exp', ascending=True) \
                                   .head(3) \
                                   .reset_index() 

short_list = short_list.entity.to_list()

In [ ]:
shorter_life = df.loc[df['entity'].isin(short_list)]

In [ ]:
shortest_life = shorter_life.groupby('entity').agg({'life_exp':'mean','total_daily_protein':'mean','daily_cal_animal_protein':'mean','daily_cal_veg_protein':'mean','gdp_per_capita':'mean','year':'count'}).reset_index().round(2)

# protein and gdp

In [ ]:
plt.figure(figsize=(12, 9
                   ))
sns.barplot(df,x='year',y='total_daily_protein',color='g',label='Toal Daily Protein Intake')
sns.barplot(df,x='year',y='gdp_per_capita',color='darkorange',alpha=0.5,label='GDP Per Capita')
plt.yscale('log')
plt.ylabel('')
plt.xticks(rotation= 30)
plt.legend(loc='upper left');

In [ ]:
plt.figure(figsize=(10, 5)) 


sns.scatterplot(data= df,x=df['gdp_per_capita'], y=df['total_daily_protein'],color='r',alpha=0.5)
sns.scatterplot(data= df,x=df['gdp_per_capita'], y=df['life_exp'],color='green',alpha=0.5)


sns.color_palette("flare", as_cmap=True)
plt.xlabel('GDP', fontsize=12)
plt.ylabel('Protein Per Capita in Grams', fontsize=12)
plt.title('GDP vs Protein per day', fontsize=14)



plt.grid(True) 
plt.tight_layout() 
plt.show();

In [ ]:
sns.set_color_codes("muted")
plt.figure(figsize=(6, 6))
sns.barplot(data=long_life.sort_values(by='life_exp',ascending=False),x='life_exp',y='entity',orient="h",color='r')

sns.set_color_codes("pastel")
sns.barplot(data=shortest_life.sort_values(by='life_exp',ascending=False),x='life_exp',y='entity',orient="h",color='g')
plt.xlabel('Life Expectancy From Birth')
plt.ylabel('');

#  top 3 countires that have eaten the MOST protein rich diets from 1990-2021?

In [ ]:
top_three_most_protein = df.groupby(['entity']) \
                                   .agg({'total_daily_protein': 'sum','life_exp':'max','gdp_per_capita': 'mean'}) \
                                   .sort_values(by='total_daily_protein', ascending=False) \
                                   .head(3)\
                                   .reset_index() 

top_three_most_protein = top_three_most_protein.sort_values(by='life_exp',ascending=False)
   
sns.set_color_codes("muted")
sns.barplot(top_three_most_protein,x = 'entity',y='total_daily_protein',hue='entity',color='g')
plt.xlabel('Country')
plt.ylabel('Daily Protein Intake');



###  what countries have eaten the LEAST grams per day of protein rich diets over these years?

In [ ]:
bottom_three_least_protein = df.groupby(['entity']) \
                                   .agg({'total_daily_protein': 'sum','life_exp':'max','gdp_per_capita': 'mean'}) \
                                   .sort_values(by='total_daily_protein', ascending=True) \
                                   .head(3) \
                                   .reset_index()                      

   
sns.barplot(bottom_three_least_protein,x = 'entity',y='total_daily_protein',hue='entity',color='g')
plt.xticks(rotation = 25);

In [ ]:
bottom_three_least_protein.sort_values(by='life_exp',ascending = False)

In [ ]:
top_and_bottom_total_daily_protein = pd.concat([top_three_most_protein,bottom_three_least_protein])


# #1 What is the correlation between the consumption of food types per capita and life expectancy across different countries, and how has this correlation changed over time?   

In [ ]:
plt.figure(figsize=(10, 5))
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
sns.regplot(
    data=df, x="total_daily_protein", y="life_exp",
    ci=99, marker=(8,2,0), color='r',line_kws=dict(color="g",))
plt.xlabel('Life Expectancy')
plt.tight_layout()
plt.ylabel('Total Daily Protein Intake');

# Do countries with a higher ratio vegetables protein consumption compared to animal protein exhibit higher life expectancies, and how does this relationship vary across different regions?   

In [ ]:
# formulating a ratio 

In [ ]:
more_meat = df.loc[df['animal_protein_ratio'] > df['veg_protein_ratio']]

more_veg = df.loc[df['veg_protein_ratio'] > df['animal_protein_ratio']]

In [ ]:

df['veg_protein_ratio'] = df['daily_cal_veg_protein']/  df['daily_cal_animal_protein']
df['animal_protein_ratio']= df['daily_cal_animal_protein']/df['daily_cal_veg_protein']

plt.figure(figsize=(12, 8))
sns.scatterplot(x= df['veg_protein_ratio'],y=df['life_exp'],color='g',label="Veg Protein")
sns.scatterplot(x= df['animal_protein_ratio'],y=df['life_exp'],color='r',label="Animal Protein")
plt.xlabel('Protein Consumption')
plt.ylabel("Life Expectancy");

In [ ]:
sns.set_color_codes("muted")
plt.figure(figsize=(10, 5)) 
sns.boxplot(more_meat,x='year',y='life_exp',color='r')
sns.boxplot(more_veg,x='year',y='life_exp',color='g',)
plt.xticks(rotation =35);

In [ ]:
# veg protein X life by region

sns.lmplot(
    data=df, x="veg_protein_ratio", y="life_exp", col="region", hue="region",
    col_wrap=2, palette="Paired", ci=None,
    height=4, scatter_kws={"s": 30, "alpha": .4})
;



In [ ]:

sns.lmplot(
    data=df, x="animal_protein_ratio", y="life_exp", col="region", hue="region",
    col_wrap=2, palette="Paired", ci=None,
    height=4, scatter_kws={"s": 30, "alpha": .2});



In [ ]:

plt.figure(figsize=(10, 5)) 
sns.scatterplot(data=df,x='animal_protein_ratio',y='life_exp',hue='region',alpha=0.9,palette='Paired')

plt.xlabel("Animal Protein Consumption")

plt.ylabel("Life Expectancy");


In [ ]:

plt.figure(figsize=(10, 5)) 
sns.scatterplot(data=df,x='veg_protein_ratio',y='life_exp',hue='region',alpha=0.9,palette='Paired')

plt.xlabel("Vegetable Protein Consumption")

plt.ylabel("Life Expectancy");


# How does the variety of food consumed (measured by the number of different food categories with significant per capita consumption) correlate with life expectancy across different countries, and does this correlation differ between high-income and low-income countries?

# what is the most popular protein source in the world?

In [ ]:
protein_sources_sum = df.iloc[:,3:11].sum().round().to_frame().reset_index()

In [ ]:
ranking_proteins = protein_sources_sum.rename(columns={protein_sources_sum.columns[0]:'protein_type',
                                                       protein_sources_sum.columns[1]:'sum_of_consumption'})

In [ ]:
ranking_proteins['rank'] = ranking_proteins['sum_of_consumption'].rank(ascending=False)

In [ ]:
ranking_proteins = ranking_proteins.sort_values(by='rank',ascending= True).reset_index(drop=True)

In [ ]:
ranking_proteins.columns

In [ ]:
sns.barplot(ranking_proteins,x='sum_of_consumption',y='protein_type',orient='h',hue='rank',legend=False)
plt.xlabel('Sum of Consumption Across All Years')
plt.ylabel('');